# Парсинг данных с сайта Московской Биржи
В данном ноутбуке будет показано, как скачивать:
* котировки акций за период
* цену и НКД облигаций за период
* котировки отраслевых индексов за период
* веса акций компаний, входящих в индекс Московской Биржи, на дату и за период



*Данный ноутбук не является индивидуальной инвестиционной рекомендацией*

In [1]:
! pip install aiomoex

In [2]:
! pip install apimoex

In [9]:
import numpy as np
import asyncio
import aiohttp
import aiomoex
import pandas as pd
import apimoex
import requests
import datetime
import pandas_datareader as pdr

## 1. Котировки акций по тикерам
Через `apimoex` происходит выгрузка котировок. Сохраняются столбцы `TRADEDATE` (дата торгов) и `CLOSE`(цена закрытия). Можно также добавить `VALUE` и `VOLUME`.

Выбор столбцов в таблице происходит в следующей строчке:

`data = apimoex.get_board_history(session, ticker)`

### Скачать котировки одной акций
Создается сессия для работы с API MOEX и загружаются исторические данные о цене акций "Газпрома". Затем эти данные преобразуются в DataFrame и устанавливается индекс по дате торгов.

In [7]:
#Пример скачивания для одной акции
with requests.Session() as session:
  data = apimoex.get_board_history(session, 'GAZP')
  df = pd.DataFrame(data)
  df.set_index('TRADEDATE', inplace = True)
  print(df.head(), '\n')
  print(df.tail(), '\n')
  df.info()

           BOARDID   CLOSE    VOLUME         VALUE
TRADEDATE                                         
2014-06-09    TQBR  144.40  40347300  5.851239e+09
2014-06-10    TQBR  144.75  41460990  5.956067e+09
2014-06-11    TQBR  146.40  39418520  5.725373e+09
2014-06-16    TQBR  145.20  77165360  1.113453e+10
2014-06-17    TQBR  144.77  44370530  6.436430e+09 

           BOARDID   CLOSE    VOLUME         VALUE
TRADEDATE                                         
2024-07-10    TQBR  117.81  99730430  1.212762e+10
2024-07-11    TQBR  121.75  72196900  8.725122e+09
2024-07-12    TQBR  119.65  48210120  5.784883e+09
2024-07-15    TQBR  119.28  40528050  4.849082e+09
2024-07-16    TQBR  124.74  93665430  1.144448e+10 

<class 'pandas.core.frame.DataFrame'>
Index: 2551 entries, 2014-06-09 to 2024-07-16
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BOARDID  2551 non-null   object 
 1   CLOSE    2533 non-null   float64
 2   VOLUME   

### Скачать котировки нескольких акций сразу

In [6]:
tickers = ["GMKN", "HYDR", "MTSS", "RTKM", "GLTR", "SNGS", "POSI"]

# Создаем пустой DataFrame для хранения данных
df_stocks = pd.DataFrame()

# Цикл для загрузки данных по каждому тикеру
for ticker in tickers: #итерируемся по тикерам, чтобы скачивать не по одному а по несколько
    with requests.Session() as session:
        data = apimoex.get_board_history(session, ticker)
        df = pd.DataFrame(data)[['TRADEDATE', 'CLOSE']] #выбираем CLOSE как цену закрытия
        df.set_index('TRADEDATE', inplace=True) #делаем дату торгов не как отдельный столбец, а индексом
        df_stocks = pd.concat([df_stocks, df], axis=1) #добавляем данные в красивую таблицу pandas

df_stocks.columns = tickers
df_stocks = df_stocks.sort_index(ascending=False) #сортировка по дню торгов от самого раннего к самому позднему

In [8]:
# Определяем начальную и конечную даты для фильтрации
start_date = '2024-06-21'
end_date = '2024-07-16'

# Фильтруем DataFrame по диапазону дат
filtered_df_stocks = df_stocks.query("index >= @start_date & index <= @end_date")

filtered_df_stocks.head(5)

,GMKN,HYDR,MTSS,RTKM,GLTR,SNGS,POSI
TRADEDATE,,,,,,,
2024-07-16,126.10,0.5865,220.85,83.75,554.45,27.375,2981.8
2024-07-15,122.76,0.5822,260.60,81.98,552.80,27.315,2929.6
2024-07-12,125.26,0.6051,270.45,84.81,540.00,28.170,3047.8
2024-07-11,126.10,0.6177,276.40,84.12,537.00,28.485,2969.2
2024-07-10,124.30,0.5970,264.70,80.20,497.45,27.350,2829.4


## 2. Цена и НКД облигаций
Для облигаций нужно знать `ISIN`, который можно также посмотреть на сайте Московской Биржи. В данном ноутбуке рассмотрим следующие облигации:


    RU000A1008J4 - АФК Система БО 001P-10
    RU000A107RZ0 - ГК Самолет БО-П13


Задаем `begin_date`, нужные ISIN в `ISIN_bonds` и также вручную задаем названия столбцов в `new_column_names`. Выбираем столбцы `ACCINT` - НКД облигации и `CLOSE` - Цена облигации.

In [22]:
#задаем ISIN облигаций
ISIN_bonds = ['RU000A1008J4', 'RU000A107RZ0']

In [23]:
begin_date = '2024-04-15'
tickers = ISIN_bonds

# Создаем пустой DataFrame для хранения данных
df_bonds = pd.DataFrame()

new_column_names = ["АФК Система БО 001P-10 Цена", "АФК Система БО 001P-10 НКД", "ГК Самолет БО-П13 Цена", "ГК Самолет БО-П13 НКД"]

for num, ticker in enumerate(tickers):
  df = pdr.moex.MoexReader(ticker, start=begin_date)
  df1 = df.read_all_boards()[["CLOSE", "ACCINT"]].dropna()
  df_bonds = df_bonds.merge(df1, how='outer', left_index=True, right_index=True)
  num=num*2
  df_bonds.columns = new_column_names[:num+2]

df_bonds = df_bonds.sort_index(ascending=False)

In [24]:
df_bonds.head(3)

,АФК Система БО 001P-10 Цена,АФК Система БО 001P-10 НКД,ГК Самолет БО-П13 Цена,ГК Самолет БО-П13 НКД
TRADEDATE,,,,
2024-07-16,89.72,29.56,95.23,3.23
2024-07-15,89.58,29.29,95.33,2.83
2024-07-12,89.61,28.48,95.18,1.62


## 3. Отраслевые индексы - котировки
На Московской Бирже котируются следующие отраслевые индексы:
* MOEXOG - нефти и газа
* MOEXEU - электроэнергетики
* MOEXTL - телекоммуникаций
* MOEXMM - металлов и добычи
* MOEXFN - финансов
* MOEXCN - потребительского сектора
* MOEXCH - химии и нефтехимии
* MOEXIT - информационных технологий
* MOEXRE - строительных компаний
* MOEXTN - транспорта

In [25]:
#перечисление отраслевых индексов
tickers_industries = ["MOEXOG", "MOEXEU", "MOEXTL", "MOEXMM", "MOEXFN", "MOEXCH", "MOEXIT", "MOEXRE", "MOEXTN"]

begin_date = '2024-04-15'
tickers = tickers_industries

# Создаем пустой DataFrame для хранения данных
df_industry = pd.DataFrame()

for ticker in tickers:
  df = pdr.moex.MoexReader(ticker, start=begin_date)
  df1 = df.read_all_boards()[["CLOSE"]]
  df_industry = pd.concat([df_industry, df1], axis=1)

df_industry.columns = tickers
df_industry = df_industry.sort_index(ascending=False)

In [26]:
df_industry.head(5)

,MOEXOG,MOEXEU,MOEXTL,MOEXMM,MOEXFN,MOEXCH,MOEXIT,MOEXRE,MOEXTN
TRADEDATE,,,,,,,,,
2024-07-17,7927.04,1678.35,1794.08,7175.14,9785.24,29891.73,3550.76,9296.09,1691.91
2024-07-16,7898.91,1676.36,1754.00,7183.36,9802.20,29971.13,3579.97,9313.87,1696.13
2024-07-15,7789.17,1662.79,1862.35,7045.25,9764.32,29937.75,3464.26,9083.32,1676.30
2024-07-12,7942.83,1690.81,1899.12,7260.51,10064.76,30606.74,3566.48,9200.26,1705.16
2024-07-11,8032.04,1708.78,1917.68,7299.78,10088.71,30570.94,3546.29,9240.77,1705.35


## 4. Веса в индексе по компаниям

Ниже приведен пример с Индексом МосБиржи, но можно сделать для любого индекса, который котируется на Московской Бирже

### Скачать на сегодняшнюю дату

In [30]:
import webbrowser
api_url='https://iss.moex.com/iss/statistics/engines/stock/markets/index/analytics/IMOEX.json'
with requests.Session() as session:
  data = apimoex.ISSClient(session, api_url).get_all()
df_tcr = pd.DataFrame(data['analytics'])
df_tcr.head(4)

,indexid,tradedate,ticker,shortnames,secids,weight,tradingsession
0,IMOEX,2024-07-17,AFKS,Система ао,AFKS,0.75,3
1,IMOEX,2024-07-17,AFLT,Аэрофлот,AFLT,0.80,3
2,IMOEX,2024-07-17,AGRO,AGRO-гдр,AGRO,0.94,3
3,IMOEX,2024-07-17,ALRS,АЛРОСА ао,ALRS,1.13,3


#### Скачать за период
Настройка параметров - `start_date` и `end_date`. Также может быть проблема с параметром `pagesize`, так как по ссылке отображается только 20 компаний из индекса МосБиржи.

Веса можно скачать за период с 2001-01-01.

In [32]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Функция для генерации списка дат
def generate_dates(start_date, end_date):
    dates = []
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)
    return dates

# URL API
api_url = 'https://iss.moex.com/iss/statistics/engines/stock/markets/index/analytics/IMOEX.json'

# Даты начала и конца периода
start_date = datetime.strptime('2024-05-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-07-16', '%Y-%m-%d')

# Генерация списка дат
dates = generate_dates(start_date, end_date)

# Создание сессии и отправка запросов
all_data = []
with requests.Session() as session:
    for date in dates:
        start = 0
        while True:
            # Параметры запроса
            params = {
                'date': date,
                'start': start,
                'pagesize': 100  # Установка размера страницы
            }

            # Отправка запроса с параметрами
            response = session.get(api_url, params=params)

            # Проверка успешности запроса
            if response.status_code == 200:
                data = response.json()
                # Проверка структуры данных
                if 'analytics' in data and 'data' in data['analytics'] and 'columns' in data['analytics']:
                    columns = data['analytics']['columns']
                    for item in data['analytics']['data']:
                        # Преобразование списка в словарь с добавлением заголовков
                        item_dict = {header: value for header, value in zip(columns, item)}
                        item_dict['date'] = date  # Добавление даты в данные
                        all_data.append(item_dict)

                    # Проверка наличия курсора для следующей страницы
                    if 'analytics.cursor' in data and len(data['analytics.cursor']['data']) > 0:
                        cursor_data = data['analytics.cursor']['data'][0]
                        total = cursor_data[1]
                        page_size = cursor_data[2]
                        start += page_size
                        if start >= total:
                            break
                    else:
                        break
                else:
                    break
            else:
                print(f"Ошибка запроса для даты {date}: {response.status_code}")
                break

# Преобразование всех данных в DataFrame
df_weights = pd.DataFrame(all_data)

# Вывод DataFrame
print(df_weights.head(4))

  indexid   tradedate ticker  shortnames secids  weight  tradingsession  \
0   IMOEX  2024-05-02   AFKS  Система ао   AFKS    0.80               3   
1   IMOEX  2024-05-02   AFLT    Аэрофлот   AFLT    0.66               3   
2   IMOEX  2024-05-02   AGRO    AGRO-гдр   AGRO    0.89               3   
3   IMOEX  2024-05-02   ALRS   АЛРОСА ао   ALRS    1.29               3   

         date  
0  2024-05-02  
1  2024-05-02  
2  2024-05-02  
3  2024-05-02  


## 5. Дополнительно

### Получить информацию по составу указанного индекса за указанную дату

Ключевая функция - `get_index_tickers`. В данной строчке можно задать любой индекс:


      data = apimoex.get_index_tickers(session, "MOEXOG")

In [28]:
with requests.Session() as session:
        data = apimoex.get_index_tickers(session, "MOEXOG")
        df = pd.DataFrame(data)
df.tail(5)

,ticker,from,till,tradingsession
18,TATNP,2007-10-15,2024-07-17,3
19,TNBP,2011-01-25,2012-09-17,3
20,TNBPP,2011-04-25,2013-10-25,3
21,TRMK,2017-12-22,2020-03-19,3
22,TRNFP,2007-04-16,2024-07-17,3
